# 建物ポリゴン代表点のGeohash値からの経年変化箇所の検出

### 使用データ

PLATEAUの東京都都心部の2023年度、2022年度データを使用。
- [3D都市モデル（Project PLATEAU）千代田区（2023年度）](https://www.geospatial.jp/ckan/dataset/plateau-13101-chiyoda-ku-2023)
- [3D都市モデル（Project PLATEAU）東京23区（2022年度）](https://www.geospatial.jp/ckan/dataset/plateau-tokyo23ku-2022)
  - いずれも、CityGML（v3）データより、東京駅付近の3次メッシュ4区画分のLOD0データをGDALのogr2ogrでGeoPackageに変換したものを利用。
  - 3次メッシュ: 53394600,53394601,53394610,53394611
  - 平面直角座標系番号: 9系 (EPSG:6677)

In [1]:
import geohash
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
# 各年度の建物データをロードし、件数を確認
gdf_2023 = gpd.read_file('../data/plateau/geopackage/13100_tokyo_2023.gpkg')
gdf_2022 = gpd.read_file('../data/plateau/geopackage/13100_tokyo_2022.gpkg')

print(f'2023年度の建物数：{gdf_2023.shape[0]}件')
print(f'2022年度の建物数：{gdf_2022.shape[0]}件')

2023年度の建物数：4625件
2022年度の建物数：4405件


In [13]:
# 代表点(重心 or ポリゴン内保証点)取得処理
def rep_point(geometry):
    centroid = geometry.centroid
    if geometry.contains(centroid):
        return centroid
    else:
        return geometry.point_on_surface()

# 2023年度データに代表点・Geohash値を追加
rep_points_2023 = gdf_2023.to_crs('EPSG:6677').geometry.apply(rep_point)
gdf_2023 = gdf_2023.assign(
    rep_point=gpd.GeoSeries(rep_points_2023, crs='EPSG:6677').to_crs('EPSG:4326'),
    geohash=lambda df: df['rep_point'].apply(lambda point: geohash.encode(point.y, point.x, 11))
)

# 2022年度データに代表点・Geohash値を追加
rep_points_2022 = gdf_2022.to_crs('EPSG:6677').geometry.apply(rep_point)
gdf_2022 = gdf_2022.assign(
    rep_point=gpd.GeoSeries(rep_points_2022, crs='EPSG:6677').to_crs('EPSG:4326'),
    geohash=lambda df: df['rep_point'].apply(lambda point: geohash.encode(point.y, point.x, 11))
)

# 結果をGeohash値でソートしてCSVに出力
gdf_2023.sort_values("geohash")[["geohash"]].to_csv("../results/tokyo_2023_python_only_geohash.csv", index=False)
gdf_2022.sort_values("geohash")[["geohash"]].to_csv("../results/tokyo_2022_python_only_geohash.csv", index=False)

In [14]:
# 2023年度データにのみ存在するGeohash値の行を抽出
only_in_gdf_2023 = gdf_2023[~gdf_2023['geohash'].isin(gdf_2022['geohash'])]

# 2022年度データにのみ存在するGeohash値の行を抽出
only_in_gdf_2022 = gdf_2022[~gdf_2022['geohash'].isin(gdf_2023['geohash'])]

print(f'2023年度データにのみ存在するGeohash値件数：{only_in_gdf_2023.shape[0]}件')
print(f'2023年度データにのみ存在するGeohash値件数：{only_in_gdf_2022.shape[0]}件')

2023年度データにのみ存在するGeohash値件数：4556件
2023年度データにのみ存在するGeohash値件数：4336件
